In [30]:
library(quantmod)
library(zoo)
library(urca)
library(stats)

In [31]:
setwd("./")

In [32]:
vols.zoo = read.zoo("sample.csv", header=TRUE, sep=",", format="%Y-%m-%d")

In [33]:
print(summary(ca.jo(vols.zoo, type="eigen", ecdet="trend", K=2, spec="transitory")))
print(summary(ca.jo(vols.zoo, type="trace", ecdet="trend", K=2, spec="transitory")))


###################### 
# Johansen-Procedure # 
###################### 

Test type: maximal eigenvalue statistic (lambda max) , with linear trend in cointegration 

Eigenvalues (lambda):
[1] 8.681531e-02 6.491141e-02 4.498019e-02 2.524265e-02 2.093684e-02
[6] 1.115009e-02 3.838508e-03 1.058603e-19

Values of teststatistic and critical values of test:

          test 10pct  5pct  1pct
r <= 6 |  3.00 10.49 12.25 16.26
r <= 5 |  8.75 16.85 18.96 23.65
r <= 4 | 16.50 23.11 25.54 30.34
r <= 3 | 19.94 29.12 31.46 36.65
r <= 2 | 35.90 34.75 37.52 42.36
r <= 1 | 52.35 40.91 43.97 49.51
r = 0  | 70.84 46.32 49.42 54.71

Eigenvectors, normalised to first column:
(These are the cointegration relations)

              EURUSD.l1    USDJPY.l1    GBPUSD.l1    USDCHF.l1     USDCAD.l1
EURUSD.l1  1.0000000000  1.000000000  1.000000000  1.000000000  1.0000000000
USDJPY.l1  0.0207768779 -3.308560787  2.529184122 -0.133789585  0.3862586788
GBPUSD.l1 -0.4452429768  0.168826928 -0.944150839  0.554988453 -0.

In [36]:
#indices <- c(1:dim(vols.zoo)[2])
pairs <- colnames(vols.zoo)
indices = index(pairs)
df <- vols.zoo
portfolios <- list()
weights <- list()
HLs <- list()
means <- list()
stds <- list()
for (i in indices) {
    for (j in indices[-c(1:i)]) {
        for (k in indices[-c(1:j)]) {
        data <- df[, pairs[c(i,j,k)]]
        johansen_eigen <- (ca.jo(data, type="eigen", ecdet="trend", K=2, spec="transitory"))
        johansen_trace <- (ca.jo(data, type="trace", ecdet="trend", K=2, spec="transitory"))
        for (d in c(rev(1:3))) {
            if ((johansen_eigen@teststat[d] > johansen_eigen@cval[d, "5pct"]) &
            (johansen_trace@teststat[d] > johansen_trace@cval[d, "5pct"])) {
                norm <- 1 
                vec_norm <- unname(johansen_eigen@V[-4,3-d+1] / norm)
                portfolio_values = data %*% vec_norm
                fit <- ar.ols(portfolio_values, order.max = 1, demean = F, intercept = T)
                theta <- -log(fit$ar)
                hl <- log(2) / theta
                portfolios <- append(portfolios, list(pairs[c(i,j,k)]))
                weights <- append(weights, list(vec_norm))
                HLs <- append(HLs, hl)
                means <- append(means, fit$x.intercept / (1-fit$ar))
                stds <- append(stds, sd(fit$resid[-1]) / sqrt(1-fit$ar^2))
            }
        }
        }
    }
}
strategies <- data.frame(Portfolio=sapply(portfolios, FUN=toString),
Weights=sapply(weights, FUN=toString),
HL = unlist(HLs),
Mean = unlist(means),
Std = unlist(stds)
)

In [37]:
strategies

Portfolio,Weights,HL,Mean,Std
<chr>,<chr>,<dbl>,<dbl>,<dbl>
"EURUSD, USDCHF, AUDUSD","1, -5.57972167393135, 2.90534678414481",17.504326,-2.96874107,1.4991862
"EURUSD, USDCHF, NZDUSD","1, -3.86151447053565, 1.75356040495089",20.736400,-2.39785099,1.0425948
"EURUSD, USDCAD, AUDUSD","1, -0.322187843980548, -0.911044455685978",17.795172,-4.76083570,0.4690320
"EURUSD, AUDUSD, NZDUSD","1, -5.70484433524699, 4.87063874545255",5.745025,-1.13909102,0.5589661
"USDJPY, USDCHF, USDCAD","1, -2.32008973886651, 1.12540474169002",10.492567,-0.58098080,0.5791998
"USDJPY, USDCHF, AUDUSD","1, -9.27351337406553, 5.56592881957701",11.763600,-0.65621467,2.3910460
"USDJPY, USDCHF, NZDUSD","1, -5.14451219765323, 2.73772191673321",11.922197,-0.06176754,1.3205588
"USDJPY, USDCAD, AUDUSD","1, 1.57019378626998, -1.8964581597152",10.693838,-0.47842252,0.5829423
"USDJPY, USDCAD, NZDUSD","1, 1.86873556171752, -2.1500013853689",9.924609,-1.24718478,0.6223840


In [29]:
write.csv(strategies, file='test.csv')